In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão de Python Neste Jupyter Notebook:', python_version())

# usaremos o filtro 'warning' para deixar mais limpo.
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

Versão de Python Neste Jupyter Notebook: 3.10.5


In [2]:
link = 'https://raw.githubusercontent.com/caiquemiranda/pandas-exercises/main/data/cars.csv'
dir = 'data/cars.csv'

#### 21. How to convert a series of date-strings to a timeseries?

In [3]:
# Input
ser = pd.Series(['01 Jan 2010', '02-02-2011',
                 '20120303', '2013/04/04',
                 '2014-05-05', '2015-06-06T12:20'])

In [4]:
# Solution 1
from dateutil.parser import parse
ser.map(lambda x: parse(x))

0   2010-01-01 00:00:00
1   2011-02-02 00:00:00
2   2012-03-03 00:00:00
3   2013-04-04 00:00:00
4   2014-05-05 00:00:00
5   2015-06-06 12:20:00
dtype: datetime64[ns]

In [5]:
# Solution 2
pd.to_datetime(ser)

0   2010-01-01 00:00:00
1   2011-02-02 00:00:00
2   2012-03-03 00:00:00
3   2013-04-04 00:00:00
4   2014-05-05 00:00:00
5   2015-06-06 12:20:00
dtype: datetime64[ns]

#### 22. How to get the day of month, week number, day of year and day of week from a series of date strings?

In [6]:
# Input
ser = pd.Series(['01 Jan 2010', '02-02-2011', 
                 '20120303', '2013/04/04',
                 '2014-05-05', '2015-06-06T12:20'])

In [7]:
# Solution
from dateutil.parser import parse
ser_ts = ser.map(lambda x: parse(x))

# day of month
print("Date: ", ser_ts.dt.day.tolist())

# week number
print("Week : ", ser_ts.dt.weekofyear.tolist())

# day of year
print("Day of year: ", ser_ts.dt.dayofyear.tolist())

Date:  [1, 2, 3, 4, 5, 6]
Week :  [53, 5, 9, 14, 19, 23]
Day of year:  [1, 33, 63, 94, 125, 157]


#### 23. How to convert year-month string to dates corresponding to the 4th day of the month?

In [8]:
# Input
ser = pd.Series(['Jan 2010', 'Feb 2011', 'Mar 2012'])

In [9]:
# Solution 1
from dateutil.parser import parse

# Parse the date
ser_ts = ser.map(lambda x: parse(x))

# Construct date string with date as 4
ser_datestr = ser_ts.dt.year.astype('str') + '-' + ser_ts.dt.month.astype('str') + '-' + '04'
# Format it.
[parse(i).strftime('%Y-%m-%d') for i in ser_datestr]

['2010-01-04', '2011-02-04', '2012-03-04']

In [10]:
# Solution 2
ser.map(lambda x: parse('04 ' + x))

0   2010-01-04
1   2011-02-04
2   2012-03-04
dtype: datetime64[ns]

#### 24. How to filter words that contain atleast 2 vowels from a series?

In [11]:
# Input
ser = pd.Series(['Apple', 'Orange', 'Plan', 'Python', 'Money'])

In [12]:
# Solution
from collections import Counter
mask = ser.map(lambda x: sum([Counter(x.lower()).get(i, 0) for i in list('aeiou')]) >= 2)
ser[mask]

0     Apple
1    Orange
4     Money
dtype: object

#### 25. How to filter valid emails from a series?

In [13]:
# Input
emails = pd.Series(['buying books at amazom.com',
                    'artui@guthn.com',
                    'aroiz@t.co',
                    'arionea@mier.com'])

In [14]:
# Solution 1 (as series of strings)
import re
pattern ='[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\\.[A-Za-z]{2,4}'
mask = emails.map(lambda x: bool(re.match(pattern, x)))
emails[mask]

1     artui@guthn.com
2          aroiz@t.co
3    arionea@mier.com
dtype: object

In [15]:
# Solution 2 (as series of list)
emails.str.findall(pattern, 
                   flags = re.IGNORECASE)

0                    []
1     [artui@guthn.com]
2          [aroiz@t.co]
3    [arionea@mier.com]
dtype: object

In [16]:
# Solution 3 (as list)
[x[0] for x in [re.findall(pattern, email) for email in emails] if len(x) > 0]

['artui@guthn.com', 'aroiz@t.co', 'arionea@mier.com']

#### 26. How to get the mean of a series grouped by another series?

In [17]:
# Input
fruit = pd.Series(np.random.choice(['apple', 'banana', 'carrot'], 10))
weights = pd.Series(np.linspace(1, 10, 10))

In [18]:
# Solution
weights.groupby(fruit).mean()

apple     4.333333
banana    6.600000
carrot    4.500000
dtype: float64

#### 27. How to compute the euclidean distance between two series?

In [19]:
# Input
p = pd.Series([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
q = pd.Series([10, 9, 8, 7, 6, 5, 4, 3, 2, 1])

In [20]:
# Solution
sum((p - q) ** 2) ** .5

18.16590212458495

In [21]:
# Solution (using func)
np.linalg.norm(p-q)

18.16590212458495

#### 28. How to find all the local maxima (or peaks) in a numeric series?

In [22]:
# Input
ser = pd.Series([2, 10, 3, 4, 9, 10, 2, 7, 3])

In [23]:
# Solution
dd = np.diff(np.sign(np.diff(ser)))
peak_locs = np.where(dd == -2)[0] + 1
peak_locs

array([1, 5, 7], dtype=int64)

#### 29.  How to replace missing spaces in a string with the least frequent character?

In [24]:
# Input
my_str = 'dbc deb abed gade'

In [25]:
# Solution
ser = pd.Series(list('dbc deb abed gade'))
freq = ser.value_counts()

print(freq)

least_freq = freq.dropna().index[-1]
"".join(ser.replace(' ', least_freq))

d    4
b    3
     3
e    3
a    2
c    1
g    1
dtype: int64


'dbcgdebgabedggade'

#### 30. How to create a TimeSeries starting ‘2000-01-01’ and 10 weekends (saturdays) after that having random numbers as values?

In [26]:
# Solution
ser = pd.Series(np.random.randint(1,10,10), 
                pd.date_range('2000-01-01', periods = 10, freq = 'W-SAT'))
ser

2000-01-01    4
2000-01-08    4
2000-01-15    5
2000-01-22    9
2000-01-29    2
2000-02-05    7
2000-02-12    6
2000-02-19    2
2000-02-26    2
2000-03-04    8
Freq: W-SAT, dtype: int32

#### 31. How to fill an intermittent time series so all missing dates show up with values of previous non-missing date?

In [27]:
# Input
ser = pd.Series([1,10,3, np.nan], 
                index=pd.to_datetime(['2000-01-01', 
                                      '2000-01-03',
                                      '2000-01-06',
                                      '2000-01-08']))

In [28]:
# Solution
ser.resample('D').ffill() # fill with previous value

2000-01-01     1.0
2000-01-02     1.0
2000-01-03    10.0
2000-01-04    10.0
2000-01-05    10.0
2000-01-06     3.0
2000-01-07     3.0
2000-01-08     NaN
Freq: D, dtype: float64

In [29]:
# Alternatives
ser.resample('D').bfill() # fill with next value
ser.resample('D').bfill().ffill() # fill next else prev value

2000-01-01     1.0
2000-01-02    10.0
2000-01-03    10.0
2000-01-04     3.0
2000-01-05     3.0
2000-01-06     3.0
2000-01-07     3.0
2000-01-08     3.0
Freq: D, dtype: float64

#### 32. How to compute the autocorrelations of a numeric series?

In [30]:
# Input
ser = pd.Series(np.arange(20) + np.random.normal(1, 10, 20))

In [31]:
# Solution
autocorrelations = [ser.autocorr(i).round(2) for i in range(11)]
print(autocorrelations[1:])
print('Lag having highest correlation: ', np.argmax(np.abs(autocorrelations[1:]))+1)

[-0.07, 0.25, 0.2, -0.49, 0.1, -0.24, 0.11, 0.55, 0.12, 0.39]
Lag having highest correlation:  8


#### 33. How to import only every nth row from a csv file to create a dataframe?

In [32]:
# Solution 1: Use chunks and for-loop
df = pd.read_csv('https://raw.githubusercontent.com/caiquemiranda/pandas-exercises/main/data/bostonhousing.csv',
                 chunksize = 50)
df2 = pd.DataFrame()

for chunk in df:
    df2 = df2.append(chunk.iloc[0,:])

In [33]:
# Solution 2: Use chunks and list comprehension
df = pd.read_csv('https://raw.githubusercontent.com/caiquemiranda/pandas-exercises/main/data/bostonhousing.csv', chunksize=50)
df2 = pd.concat([chunk.iloc[0] for chunk in df], 
                axis = 1)

df2 = df2.transpose()

In [34]:
# Solution 3: Use csv reader
import csv

with open('data/bostonhousing.csv', 'r') as f:
    reader = csv.reader(f)
    out = []

    for i, row in enumerate(reader):
        if i%50 == 0:
            out.append(row)

df2 = pd.DataFrame(out[1:], columns=out[0])
print(df2.head())

      crim  zn  indus chas    nox     rm   age     dis rad  tax ptratio  \
0  0.21977   0   6.91    0  0.448  5.602    62  6.0877   3  233    17.9   
1   0.0686   0   2.89    0  0.445  7.416  62.5  3.4952   2  276      18   
2  2.73397   0  19.58    0  0.871  5.597  94.9  1.5257   5  403    14.7   
3   0.0315  95   1.47    0  0.403  6.975  15.3  7.6534   3  402      17   
4  0.19073  22   5.86    0  0.431  6.718  17.5  7.8265   7  330    19.1   

        b  lstat  medv  
0   396.9   16.2  19.4  
1   396.9   6.19  33.2  
2  351.85  21.45  15.4  
3   396.9   4.56  34.9  
4  393.74   6.56  26.2  


#### 34. How to change column values when importing csv to a dataframe?

In [35]:
# Solution 1: Using converter parameter
df = pd.read_csv('https://raw.githubusercontent.com/caiquemiranda/pandas-exercises/main/data/bostonhousing.csv',
                 converters = {'medv': lambda x: 'High' if float(x) > 25 else 'Low'})

In [36]:
# Solution 2: Using csv reader
import csv

with open('data/bostonhousing.csv', 'r') as f:
    reader = csv.reader(f)
    out = []

    for i, row in enumerate(reader):
        if i > 0:
            row[13] = 'High' if float(row[13]) > 25 else 'Low'
            out.append(row)

df = pd.DataFrame(out[1:], columns=out[0])
print(df.head())

   0.00632 18  2.31  0  0.538  6.575  65.2    4.09  1  296  15.3   396.9  \
0  0.02731  0  7.07  0  0.469  6.421  78.9  4.9671  2  242  17.8   396.9   
1  0.02729  0  7.07  0  0.469  7.185  61.1  4.9671  2  242  17.8  392.83   
2  0.03237  0  2.18  0  0.458  6.998  45.8  6.0622  3  222  18.7  394.63   
3  0.06905  0  2.18  0  0.458  7.147  54.2  6.0622  3  222  18.7   396.9   
4  0.02985  0  2.18  0  0.458   6.43  58.7  6.0622  3  222  18.7  394.12   

   4.98   Low  
0  9.14   Low  
1  4.03  High  
2  2.94  High  
3  5.33  High  
4  5.21  High  


#### 35. How to create a dataframe with rows as strides from a given series?

In [37]:
# Input
L = pd.Series(range(15))

In [38]:
def gen_strides(a, stride_len=5, window_len=5):
    n_strides = ((a.size-window_len)//stride_len) + 1
    return np.array([a[s:(s+window_len)] for s in np.arange(0, a.size, stride_len)[:n_strides]])

gen_strides(L, stride_len = 2, window_len = 4)

array([[ 0,  1,  2,  3],
       [ 2,  3,  4,  5],
       [ 4,  5,  6,  7],
       [ 6,  7,  8,  9],
       [ 8,  9, 10, 11],
       [10, 11, 12, 13]], dtype=int64)

#### 36. How to import only specified columns from a csv file?

In [39]:
df = pd.read_csv('https://raw.githubusercontent.com/caiquemiranda/pandas-exercises/main/data/bostonhousing.csv',
                 usecols = ['crim', 'medv'])
print(df.head())

      crim  medv
0  0.00632  24.0
1  0.02731  21.6
2  0.02729  34.7
3  0.03237  33.4
4  0.06905  36.2


#### 37. How to get the nrows, ncolumns, datatype, summary stats of each column of a dataframe? Also get the array and list equivalent.

In [40]:
df = pd.read_csv('https://raw.githubusercontent.com/caiquemiranda/pandas-exercises/main/data/cars.csv')

In [41]:
# number of rows and columns
print(df.shape)

(93, 27)


In [42]:
# datatypes
print(df.dtypes)

Manufacturer           object
Model                  object
Type                   object
Min.Price             float64
Price                 float64
Max.Price             float64
MPG.city              float64
MPG.highway           float64
AirBags                object
DriveTrain             object
Cylinders              object
EngineSize            float64
Horsepower            float64
RPM                   float64
Rev.per.mile          float64
Man.trans.avail        object
Fuel.tank.capacity    float64
Passengers            float64
Length                float64
Wheelbase             float64
Width                 float64
Turn.circle           float64
Rear.seat.room        float64
Luggage.room          float64
Weight                float64
Origin                 object
Make                   object
dtype: object


In [43]:
# summary statistics
df_stats = df.describe()

In [44]:
# numpy array
df_arr = df.values

In [45]:
# list
df_list = df.values.tolist()

#### 38. How to extract the row and column number of a particular cell with given criterion?

In [46]:
# Input
df = pd.read_csv('https://raw.githubusercontent.com/caiquemiranda/pandas-exercises/main/data/cars.csv')

In [ ]:
# Solution
# Get Manufacturer with highest price
df.loc[df.Price == np.max(df.Price), ['Manufacturer', 'Model', 'Type']]

# Get Row and Column number
row, col = np.where(df.values == np.max(df.Price))

# Get the value
df.iat[row[0], col[0]]
df.iloc[row[0], col[0]]

# Alternates
df.at[row[0], 'Price']
df.values(row[0], 'Price')

# The difference between `iat` - `iloc` vs `at` - `loc` is:
# `iat` snd `iloc` accepts row and column numbers.
# Whereas `at` and `loc` accepts index and column names.

#### 39. How to rename a specific columns in a dataframe?

In [48]:
# Input
df = pd.read_csv('https://raw.githubusercontent.com/caiquemiranda/pandas-exercises/main/data/cars.csv')

In [49]:
# Solution
# Step 1:
df=df.rename(columns = {'Type':'CarType'})
# or
df.columns.values[2] = "CarType"

In [50]:
# Step 2:
df.columns = df.columns.map(lambda x: x.replace('.', '_'))
print(df.columns)

Index(['Manufacturer', 'Model', 'CarType', 'Min_Price', 'Price', 'Max_Price',
       'MPG_city', 'MPG_highway', 'AirBags', 'DriveTrain', 'Cylinders',
       'EngineSize', 'Horsepower', 'RPM', 'Rev_per_mile', 'Man_trans_avail',
       'Fuel_tank_capacity', 'Passengers', 'Length', 'Wheelbase', 'Width',
       'Turn_circle', 'Rear_seat_room', 'Luggage_room', 'Weight', 'Origin',
       'Make'],
      dtype='object')


#### 40. How to check if a dataframe has any missing values?

In [51]:
# Input
df = pd.read_csv('https://raw.githubusercontent.com/caiquemiranda/pandas-exercises/main/data/cars.csv')

In [52]:
# Solution
df.isnull().values.any()

True

In [53]:
%reload_ext watermark
%watermark -a "Caique Miranda" -gu "caiquemiranda" -iv

Author: Caique Miranda

Github username: caiquemiranda

numpy : 1.23.0
pandas: 1.4.3
re    : 2.2.1
csv   : 1.0

